In [ ]:
%% Segunda parte
% 3
C = [1 0 0 0]

% Matriz observavel
Ob = [C; C*A; C*A*A; C*A*A*A]
rank(Ob)

C = [1 0 0 0]

Ob = [C; C*A; C*A*A; C*A*A*A]
Ob_rank = rank(Ob)

% rank(Ob) = 2
% Portanto para a saída C = [1 0 0 0] o sistema não será observável

% 4

C = [0 0 1 0]

Ob = [C; C*A; C*A*A; C*A*A*A]
Ob_rank = rank(Ob)

% Para a saída C = [ 0 0 1 0] o sistema será observável, rank(Ob) = 4







% 5 projetando o observador para esse sistema com saída C = [0 0 1 0]

% Forma canônica observador
Ob_inv = inv(Ob)
Q = [Ob_inv(1,4); Ob_inv(2,4); Ob_inv(3,4); Ob_inv(4,4);] 

T = [Q A*Q A*A*Q A*A*A*Q]

Ao = inv(T)*A*T
Bo = inv(T)*B
Co = C*T

syms L0 L1 L2 L3 s
% L = [L0; L1; L2; L3]
% M = (Ao-L*Co)

% Polinômio característico: s^4 - (-L3)*s^3 - (16.0976-L2)*s^2 - (-L1)*s - (-L0)
% Polinômio característico: s^4 + L3*s^3 + (L2 - 16.0976)*s^2 + L1*s + L0
% Para determinar os polos desejados precisamos encontrar as raízes do
% controlador e escolher uma que seja entre 4 e 10x mais rápida para o
% observador, portanto

% P = [-.8532 -1.0532 -1.2532 -1.4532]

% L = place(A',C',P)'
sys_r = ss(A,B,C,0)

Nbar = rscale(sys_r,K)

Ace = [(A-B*K) (B*K);
       zeros(size(A)) (A-L*C)];
Bce = [B*Nbar;
       zeros(size(B))];
Cce = [C zeros(size(C))];
Dce = [0];

states = {'x' 'x_dot' 'phi' 'phi_dot' 'e1' 'e2' 'e3' 'e4'};
inputs = {'r'};
outputs = {'Cart position(m)'};

sys_cl = ss(Ace,Bce,Cce,Dce,'statename',states,'inputname',inputs,'outputname',outputs);
x0 = [0.1; 0; 1; 0; 0; 0; 0; 0]
t = 0:0.01:50;
r = 0.2*ones(size(t));
[y,t,x]=lsim(sys_cl,r,t,x0')


figure(5)
step(sys_cl)               % Step response
grid on

figure(6)
initial(sys_cl,x0)         % CI diferentes de 0
grid on

figure(10);
subplot(221)
plot(t,y); 
grid 
hold off;
title('Output ---> state x1')
subplot(222)
x_hat=-(x(:,5:8)-x(:,1:4));
plot(t,x(:,1:4),'k');hold on; plot(t,x_hat(:,1:4),'r--')
title('States x1 & x2 & x3 & x4, x1_{hat} & x2_{hat} & x3_{hat} & x4_{hat}')
grid

% x_hat=-(x(:,1:2)-x(:,5:6));
% plot(t,x(:,1:2),'k');hold on; plot(t,x_hat(:,1:2),'r--'); plot(t,x(:,3:4),'m');hold on; plot(t,x_hat(:,3:4),'g--');
% title('States x1 & x2 & x3 & x4, x1_{hat} & x2_{hat} & x3_{hat} & x4_{hat}')

hold off;
subplot(223)
plot(t,K*x_hat');
title('Control signal u = K.x_{hat}')
grid
xlim([0 10])
hold off;


figure(7)
[u,t] = gensig("square", 90, 180)
lsim(-sys_cl,u,t)                    % Step with disturbance
grid on





% figure(8)
% [AX,H1,H2] = plotyy(t,y(:,1),t,y(:,2),'plot');
% set(get(AX(1),'Ylabel'),'String','cart position (m)')
% set(get(AX(2),'Ylabel'),'String','pendulum angle (radians)')
% title('Step Response with Observer-Based State-Feedback Control')
% grid on